In [7]:
import getpass
import os

try:
    # load environment variables from .env file (requires `python-dotenv`)
    from dotenv import load_dotenv

    load_dotenv()
except ImportError:
    pass


from pydantic_ai import Agent
from pydantic_ai import Agent
from pydantic_ai.models.openai import OpenAIChatModel
from pydantic_ai.providers.azure import AzureProvider

model = OpenAIChatModel("gpt-4.1-mini", provider=AzureProvider())

agent = Agent(
    model=model,
    instructions="You are an expert when it comes to writing complete and detailed docstrings from not very informative tool definitions." \
    "Your task is to improve the tool descriptions by infering their intended use from the context and from the parameter json schema." \
    "All these tools are used for the Ally Portal, which is a web application to manage Copilot configurations." \
    "Make sure the docstrings follow standard formating guidelines." \
    "This is important: Only answer with the new docstring, do not include any other text!", 
)




In [8]:
from lib.openapi_to_tools import OpenAPIToolsLoader

# Initialize the loader
loader = OpenAPIToolsLoader(
    openapi_url="https://ally-config-ui.dev.copilot.aws.inform-cloud.io/openapi.json"
)

# Load all tools
print("Loading tools from OpenAPI spec...")
tools = loader.load_tools()

Loading tools from OpenAPI spec...
Created tool: get_aws_logs_api_getAWSLogs_post [POST /api/getAWSLogs]
Created tool: get_capabilities_api_capabilities_get [GET /api/capabilities]
Created tool: get_active_config_api_getActiveConfig_post [POST /api/getActiveConfig]
Created tool: validate_config_api_validateConfig_post [POST /api/validateConfig]
Created tool: set_config_api_setConfig_post [POST /api/setConfig]
Created tool: get_config_history_api_getConfigHistory_post [POST /api/getConfigHistory]
Created tool: create_endpoint_api_createEndpoint_post [POST /api/createEndpoint]
Created tool: get_endpoint_metadata_api_getEndpointMetadata_post [POST /api/getEndpointMetadata]
Created tool: update_endpoint_metadata_api_updateEndpointMetadata_post [POST /api/updateEndpointMetadata]
Created tool: delete_endpoint_api_deleteEndpoint_post [POST /api/deleteEndpoint]
Created tool: get_endpoints_api_getEndpoints_post [POST /api/getEndpoints]
Created tool: get_endpoint_authorization_api_endpoint_autho

In [9]:
for tool in tools:
    tool_def=tool.tool_def
    new_docstring=await agent.run(f"Generate a new improved docstring for me:\n\n{tool_def}")
    result = new_docstring.output.strip()
    print(f"New docstring for tool {tool.name}:\n{result}\n")

New docstring for tool get_aws_logs_api_getAWSLogs_post:
"""
Retrieve log data from a specified AWS CloudWatch log group within a given time range.

This function fetches logs from the specified AWS log group, optionally filtering by a session ID to narrow down the logs related to a particular user session or transaction. The retrieved logs cover the period between the provided start and end timestamps.

Parameters:
    log_group (str): The name of the AWS CloudWatch log group from which to retrieve logs.
    session_id (str or None): Optional session ID to filter logs related to a specific session. If None, logs for all sessions are retrieved.
    start_time (str): The start timestamp (ISO 8601 or compatible format) indicating the beginning of the log retrieval interval.
    end_time (str): The end timestamp (ISO 8601 or compatible format) indicating the end of the log retrieval interval.

Returns:
    dict: A response object containing:
        - A list of log entries matching the qu

In [10]:
# Collect improved docstrings and write to markdown file
improved_docstrings = []

for tool in tools:
    tool_def = tool.tool_def
    new_docstring = await agent.run(f"Generate a new improved docstring for me:\n\n{tool_def}")
    result = new_docstring.output.strip()
    
    improved_docstrings.append({
        'name': tool.name,
        'docstring': result
    })
    
    print(f"Processed tool: {tool.name}")

print(f"\nProcessed {len(improved_docstrings)} tools total.")

Processed tool: get_aws_logs_api_getAWSLogs_post
Processed tool: get_capabilities_api_capabilities_get
Processed tool: get_active_config_api_getActiveConfig_post
Processed tool: validate_config_api_validateConfig_post
Processed tool: set_config_api_setConfig_post
Processed tool: get_config_history_api_getConfigHistory_post
Processed tool: create_endpoint_api_createEndpoint_post
Processed tool: get_endpoint_metadata_api_getEndpointMetadata_post
Processed tool: update_endpoint_metadata_api_updateEndpointMetadata_post
Processed tool: delete_endpoint_api_deleteEndpoint_post
Processed tool: get_endpoints_api_getEndpoints_post
Processed tool: get_endpoint_authorization_api_endpoint_authorization_get
Processed tool: update_endpoint_authorization_api_endpoint_authorization_post
Processed tool: delete_endpoint_authorization_api_endpoint_authorization_delete
Processed tool: execute_evaluation_suite_api_executeEvaluationSuite_post
Processed tool: get_stored_evaluation_results_api_getStoredEvaluat

In [ ]:
# Write improved docstrings to markdown file
markdown_content = "# Improved Tool Docstrings\n\n"
markdown_content += "This document contains improved docstrings for tools generated from the Ally Portal OpenAPI specification.\n\n"
markdown_content += "---\n\n"

for item in improved_docstrings:
    markdown_content += f"## {item['name']}\n\n"
    markdown_content += f"```\n{item['docstring']}\n```\n\n"
    markdown_content += "---\n\n"

# Write to file
with open("ally_config_improved_tool_descriptions.md", "w") as f:
    f.write(markdown_content)

print("✅ Markdown file 'improved_tool_descriptions.md' has been created successfully!")
print(f"📄 Contains improved docstrings for {len(improved_docstrings)} tools.")

✅ Markdown file 'improved_tool_descriptions.md' has been created successfully!
📄 Contains improved docstrings for 36 tools.
